https://www.youtube.com/watch?v=8rXD5-xhemo&list=PLoROMvodv4rOhcuXMZkNm7j3fVwBBY42z&index=2&t=2433s

https://medium.com/data-science-group-iitr/word-embedding-2d05d270b285

# The goal of this course: to represent the meaning of words

- WordNet library is an okay to process words using computers:
    - extract synonym sets and hypernyms ('is a; relationships)
- Wordnet however requires human labor to create and adapt => lot of missing words, some incorrect word synonyms or interpretation.

- Bag-of-words (one-hot representation) cannot capture similarity between words (since each word vector are orthogonal, aka dot product is 0, aka different position of '1' => no natural notion of similarity
- WordNet can help, but it has its cons (see above)


# Distributional semantics

Because of all the cons above, **vector needs to learn to encode the similarity themselves**

- Word meaning is given by its context: other words that frequently appear close-by (position before and after)
- With this, you can represent a word using a vector with positive/negative values(not one-hot), aka word vectors, aka word embeddings. Length of these vectors can go from 50 to 300, depends on the computational resources.

## Word2Vec skip-gram

Idea:
- for each word in a text (center word c), look at the context words/neighbor words (o) around c.
- Calculate **P(o|c)** using **word vectors' similarity** for c and o
    - The idea is to optimize/update this word vector matrix (parameters) using gradient descent on some loss function (below) so that the **CORRECT NEIGHBOR WORD VECTORS (related to a given center word) are updated in a way that maximize the probability** 

![](images/word2vec.png)

## The loss function

![](images/w2v_formula.png)

**Note that word vectors matrix is the variable delta** (the only variable to be optimized)

### Explain the loss function and P(o|c) (aka P(neighbor|center)

![](images/w2v_loss.png)

![](images/w2v_prob_formula.png)

**Conclusion**: if neighbor word (n) and center word (c) have **high similarity (from dot product OF TWO DIFFERENT VERSION**: **neighbor (NOT CENTER) version of neighbor word** and **center version of center word)** => they are **close together in the vector space aka they have similar meaning**, P(n|c) is big.

Why 2 versions? See below


In [ ]:
# (because 2 center version of 2 words should be similar in meaning, 
# like king and prince, instead of how good of neighbors they can be): 
# THIS MIGHT BE WRONG

in another word, **Softmax is used on the dot product to calculate P(n|c)**: bigger dot product, higher softmax => higher P(n|c) 

As an effect, by updating word vectors so that similar words are nearby in space, w2v model can minimize the objective function

**To export your word vectors (1 matrix), you can just average these 2 versions**

## Train model and update word vector matrix 

Note that **we have two matrices (neighbor version and center version) to update**, each has shape: vocab_size x # of dimension

To optimize these matrices (parameters), **we use gradient descent**. **The only input for this model is the parameters themselves** (which is somewhat similar to collab filtering)

### Formula for derivative of loss function

https://youtu.be/8rXD5-xhemo?t=4316

or: derivative of P(n|c) = nn - sum-w-in-vocab( P(w|c) ) * wn

Notice that this derivative is fairly easy to calculate, thus more practical, **because we use 2 matrices: center and neighbor**

Note: what happen if we only use 1 version (1 matrix)? There will be instances where the same word vector got multiplied => square term => hard to take derivative of 

## Skip-gram model with negative sampling 

**Note**: pay attention to the denominator of P(n|c) above, and you can see we are adding the entire vocabulary in order to calculate probability of a set of center word and neighbor word => extremely slow to go through the corpus with this calculation

=> Solution: **negative sampling**: train **binary logistic regression** (TODO: binary? with the way sigmoid is used, this is more like multiclass classification) for:
- a **true pair** (neighbor word and center word). We want this probability (after applying sigmoid) to be high
- **some negative samples**: noise pairs (center word paired with random word). We want THESE probabilities to be low

Example: Started one sentence, and replaces 1 word with another (‘cats are cute’ to ‘justice are cute’). Then they made labels 1 if it was unchanged, and 0 if it was changed. Then train a binary logistic regression model and update word vector matrix so that **true pair** probability is higher than **negative samples'**

![](images/Selection_028.png)

That is a **sigmoid function** as in sigmoid(uoT @ vc).

**Question on the loss function:**
- How the sample K words (the negative sample)? => start with unigram distribution

- Why the negative in the dot product before sigmoid (in K sum)? The negative before sigmoid is equivalent to  1 - probability, so as we want this dot product of the negative sample to be small => probability (b/c of neg and sigmoid) will be big => log small. 
    - And for the first term (for the real pair), we want big dot product => big sigmoid => big probability so that log will close to 0 (small). Remember that I am describing best case scenario for these 2 terms, plus that our objective is to minimize this loss! 

in P(w) = U(w)\**3/4 / Z
- U(w) is the unigram count of word w
- Z: normalization term, so that P(w) will be a probability
- Why the 3/4? Explained in the pic (last line)

## Continuous bag of words (CBOW)

The opposite of skip-gram: Use all of the context words (independently, like Naive Bayes) to predict the center words